# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = pd.read_csv("../output_data/city.csv")
weather_file


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Rikitea,91,PF,1619822594,78,-23.1203,-134.9692,78.78,20.40
1,1,Takapau,76,NZ,1619822594,75,-40.0333,176.3500,55.99,8.03
2,2,Stonewall,90,CA,1619822595,70,50.1344,-97.3268,45.00,20.71
3,3,Narsaq,8,GL,1619822595,30,60.9167,-46.0500,51.80,27.63
4,4,Busselton,99,AU,1619822595,98,-33.6500,115.3333,51.01,5.44
...,...,...,...,...,...,...,...,...,...,...
120,120,Dzhebariki-Khaya,100,RU,1619822630,97,62.2167,135.8000,34.63,1.92
121,121,Cutro,99,IT,1619822631,42,39.0353,16.9819,70.00,8.52
122,122,Saint George,1,US,1619822504,9,37.1041,-113.5841,93.00,10.36
123,123,Teya,100,RU,1619822631,91,60.3778,92.6267,35.78,12.62


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [5]:
hfigure = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
hfigure.add_layer(heat_layer)
#Display figure
hfigure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
possible_vacay = weather_file.loc[(weather_file["Wind Speed"] <= 10) & (weather_file["Cloudiness"] == 0) & \
                                   (weather_file["Max Temp"] >= 70) & (weather_file["Max Temp"] <= 80)].dropna()

possible_vacay

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
27,27,Igarapava,0,BR,1619822602,69,-20.0383,-47.7469,75.2,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
possible_hotels = possible_vacay.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
possible_hotels["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [8]:
for index, row in possible_hotels.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        possible_hotels.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
possible_hotels

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.0439869,
                    "lng": -47.763196
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.04257727010728,
                        "lng": -47.76179197010728
                    },
                    "southwest": {
                        "lat": -20.04527692989272,
                        "lng": -47.76449162989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "\u00c2ngelo Palace Hotel Igarapava",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2592,
                    "html_attributions": [
       

,City,Country,Lat,Lng,Hotel Name
27,Igarapava,BR,-20.0383,-47.7469,Ângelo Palace Hotel Igarapava


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in possible_hotels.iterrows()]
locations = possible_hotels[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

hfigure.add_layer(markers)

# Display Map
hfigure


Figure(layout=FigureLayout(height='420px'))